In [1]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [5]:
import gzip
import simplejson as json

In [6]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [7]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [8]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [9]:
len(scripts) / 2

191363.0

In [10]:

def describe(key):
    key_list = sorted([script[key] for script in scripts])
    l = len(key_list)
    total = sum(key_list)
    avg = total / l
    s = (sum([(k - avg)**2 for k in key_list]) / l)**0.5
    med = (key_list[l // 2 - 1] + key_list[l // 2]) / 2
    q25 = key_list[l // 4]
    q75 = key_list[l // 4 + l // 2]

    return (total, avg, s, q25, med, q75)

In [11]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]
print(summary)

[('items', (4410054, 11.522744731217633, 33.11216633979208, 1, 3.0, 8)), ('quantity', (316356836, 826.5883059943667, 3872.1810146090156, 30, 120.0, 466)), ('nic', (29048309.790000085, 75.89844899484248, 197.57282662775233, 7.7, 22.62, 65.94)), ('act_cost', (27053937.60000044, 70.68748295125087, 183.267318953028, 7.25, 21.24, 61.53))]


In [12]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [13]:
bnf_names = set([script['bnf_name'] for script in scripts])
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [14]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    groups[script['bnf_name']].append(script)

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [15]:
item_sum = []
for name in bnf_names:
    item_sum.append((name, sum([script['items'] for script in groups[name]])))
item_sum.sort(key=lambda tup: tup[1], reverse=True)
max_item = [item_sum[0]]
print(max_item)

[('Omeprazole_Cap E/C 20mg', 113826)]


**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [16]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [17]:
def group_by_field(data, field):
    field_values = set([record[field] for record in data])
    groups = {value: [] for value in field_values}
    for record in data:
        groups[record[field]].append(record)
    return groups

In [18]:
groups = group_by_field(scripts, 'bnf_name')
test_max_item = max(groups, key=lambda group: sum([script['items'] for script in groups[group]]))
print(test_max_item)
# assert test_max_item == max_item

Omeprazole_Cap E/C 20mg


## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [19]:
practices.sort(key=lambda dict: dict['post_code'])

practice_postal = {}
for practice in practices:
    if practice['code'] not in practice_postal:
        practice_postal[practice['code']] = practice['post_code']

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [20]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [21]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]
print(joined[:10])

[{'bnf_code': '0101010G0AAABAB', 'items': 2, 'practice': 'N81013', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'nic': 5.98, 'act_cost': 5.56, 'quantity': 1000, 'post_code': 'SK11 6JL'}, {'bnf_code': '0101021B0AAAHAH', 'items': 1, 'practice': 'N81013', 'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F', 'nic': 1.95, 'act_cost': 1.82, 'quantity': 500, 'post_code': 'SK11 6JL'}, {'bnf_code': '0101021B0AAALAL', 'items': 12, 'practice': 'N81013', 'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F', 'nic': 64.51, 'act_cost': 59.95, 'quantity': 6300, 'post_code': 'SK11 6JL'}, {'bnf_code': '0101021B0AAAPAP', 'items': 3, 'practice': 'N81013', 'bnf_name': 'Sod Alginate/Pot Bicarb_Tab Chble 500mg', 'nic': 9.21, 'act_cost': 8.55, 'quantity': 180, 'post_code': 'SK11 6JL'}, {'bnf_code': '0101021B0BEADAJ', 'items': 6, 'practice': 'N81013', 'bnf_name': 'Gaviscon Infant_Sach 2g (Dual Pack) S/F', 'nic': 28.92, 'act_cost': 26.84, 'quantity': 90, 'post_code': 'SK11 6JL'}, {'bnf_code': '0101021B0BEAIAL', 'i

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [22]:
groups = group_by_field(joined, 'post_code')

items_by_post = [(post, sum([script['items'] for script in groups[post]])) for post in groups.keys()]

In [23]:
postal_totals = sorted([i for i in items_by_post if i[1] > 0])[:100]

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [31]:
total_items_by_bnf_post = {post: [] for post in groups}
for post in groups:
    bnf_groups = group_by_field(groups[post], 'bnf_name')
    for bnf in bnf_groups:
        total_items_bnf = sum([script['items'] for script in bnf_groups[bnf]])
        total_items_by_bnf_post[post].append((bnf, total_items_bnf))
    
print(total_items_by_bnf_post['WN3 5HL'])

[('Systane Hydroxypropyl Guar Eye Dps 10ml', 5), ('Microgynon 30_Tab', 50), ('Typherix_Vac 0.5ml Pfs', 18), ('Clotrimazole_Pess 500mg/Crm 2%', 10), ('Benzoyl Per_Gel 5%', 11), ('Ranolazine_Tab 750mg M/R', 2), ('Metformin HCl_Tab 1g M/R', 43), ('Balneum Plus_Bath Oil', 4), ('E45_Crm', 39), ('Convatec_Orahesive Pdr', 3), ('Pioglitazone HCl_Tab 30mg', 10), ('Paracet_Tab 500mg', 616), ('Lyrica_Cap 150mg', 34), ('Cetraben Crm 500g', 18), ('Half Securon SR_Tab 120mg', 2), ('Ins Humalog Mix 25_KwikPen 100u/ml 3mlPf', 3), ('Adizem-SR_Cap 90mg', 7), ('Apixaban_Tab 2.5mg', 16), ('Paracet_Tab Solb 500mg', 19), ('Salbutamol_Inh Soln 2.5mg/2.5ml Ud', 8), ('Betameth Val/Fusidic Acid_Crm 0.1%/2%', 7), ('3m Health Care_Cavilon Durable Barrier C', 4), ('Sulfasalazine_Tab 500mg', 12), ('Lyrica_Cap 100mg', 13), ('Transtec_T/Derm Patch 35mcg/hr (20mg)', 3), ('Bumetanide_Tab 1mg', 25), ('Amoxicillin_Cap 500mg', 237), ('Rigevidon_Tab', 29), ('Eumovate_Crm 0.05%', 4), ('Fostair_Inh 200mcg/6mcg (120D) CFF', 1

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [39]:
total_items_by_post = {}
for post in groups:
    total_items = sum([script['items'] for script in groups[post]])
    total_items_by_post[post] = total_items
    
assert len(total_items_by_post) == 118
total_items_by_post

{'WN3 5HL': 31149,
 'BB7 2JG': 44585,
 'TS24 7PW': 58207,
 'LN2 2JP': 46173,
 'TW13 4GU': 19584,
 'SK11 6JL': 110071,
 'BL9 0SN': 35275,
 'NE37 2PU': 57500,
 'TW3 3LN': 40141,
 'WD18 0JP': 21034,
 'LS9 9EF': 48051,
 'SS13 3HQ': 38513,
 'BD3 8QH': 21010,
 'NW10 8RY': 21553,
 'S63 9EH': 34787,
 'LA1 1PN': 47335,
 'TN24 0GP': 16955,
 'M30 0NU': 25597,
 'YO16 4LZ': 52032,
 'SR5 2LT': 44895,
 'ST8 6AG': 34516,
 'GU9 9QS': 32131,
 'ST3 6AB': 38705,
 'TS10 4NW': 45161,
 'DN22 7XF': 43091,
 'WN7 1HR': 29076,
 'RM3 9SU': 25476,
 'LE10 1DS': 49335,
 'DN34 4GB': 48013,
 'WA9 1LN': 29644,
 'S65 1DA': 71683,
 'PL7 1AD': 65051,
 'BB9 7SR': 38224,
 'BH23 3AF': 32545,
 'NG7 3GW': 17698,
 'B23 6DJ': 24859,
 'KT6 6EZ': 38975,
 'L31 0DJ': 32065,
 'CH65 6TG': 25090,
 'CW7 1AT': 43164,
 'HA3 7LT': 32113,
 'WN7 2PE': 23130,
 'WA7 1AB': 41314,
 'GL1 3PX': 38120,
 'ST1 4PB': 24227,
 'SS9 5UU': 26095,
 'OL11 1DN': 21567,
 'ME8 8AA': 25257,
 'KT14 6DH': 26758,
 'BB4 5SL': 29388,
 'SE1 6JP': 45030,
 'WS3 3JP': 3

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [45]:
from operator import itemgetter
max_item_by_post = []
for post in total_items_by_bnf_post:
    total_items_by_bnf_post[post].sort(key=itemgetter(1, 0), reverse=True)
    max_item_by_post.append((post, total_items_by_bnf_post[post][0][0], total_items_by_bnf_post[post][0][1]))

item_by_post = []
for post in max_item_by_post:
    item_by_post.append((post[0], post[1], post[2] / total_items_by_post[post[0]]))
    
print(item_by_post[:10])

[('WN3 5HL', 'Omeprazole_Cap E/C 20mg', 0.046486243539118434), ('BB7 2JG', 'Omeprazole_Cap E/C 20mg', 0.029471795446899183), ('TS24 7PW', 'Paracet_Tab 500mg', 0.03858642431322693), ('LN2 2JP', 'Omeprazole_Cap E/C 20mg', 0.036189981157819504), ('TW13 4GU', 'Omeprazole_Cap E/C 20mg', 0.03073937908496732), ('SK11 6JL', 'Omeprazole_Cap E/C 20mg', 0.029244760200234393), ('BL9 0SN', 'Omeprazole_Cap E/C 20mg', 0.03257264351523742), ('NE37 2PU', 'Paracet_Tab 500mg', 0.02777391304347826), ('TW3 3LN', 'Omeprazole_Cap E/C 20mg', 0.02742831518895892), ('WD18 0JP', 'Omeprazole_Cap E/C 20mg', 0.032471237044784634)]


In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [46]:
items_by_region = sorted(item_by_post)[:100]
print(items_by_region)

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.03415082474725487), ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.02926161780958897), ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03549462254750369), ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.024095900880968663), ('B70 7AW', 'Paracet_Tab 500mg', 0.0266896608360023), ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 0.02884503434625684), ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 0.03645501521908402), ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 0.03428477088454342), ('BB4 5SL', 'Omeprazole_Cap E/C 20mg', 0.040696883081529876), ('BB7 2JG', 'Omeprazole_Cap E/C 20mg', 0.029471795446899183), ('BB8 0JZ', 'Atorvastatin_Tab 20mg', 0.022563442442074293), ('BB9 7SR', 'Omeprazole_Cap E/C 20mg', 0.023833193804939305), ('BD3 8QH', 'Atorvastatin_Tab 40mg', 0.03422179914326511), ('BH18 8EE', 'Omeprazole_Cap E/C 20mg', 0.029000583563798747), ('BH23 3AF', 'Omeprazole_Cap E/C 20mg', 0.03733292364418497), ('BL1 8TU', 'Omeprazole_Cap E/C 20mg', 0.03095821215368

In [47]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*